In [1]:

%matplotlib inline
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics
import pdf2docx
import csv
import pickle
from openpyxl import load_workbook

import typing
import io
import numpy as np
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
%matplotlib widget
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format=u'%(filename)+13s [ LINE:%(lineno)-4s] %(levelname)-8s %(message)s',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)

from docx2python import docx2python

import xlrd
import traceback



In [18]:
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\test.xlsx"

df = pd.read_excel(file)

df = df.applymap(str)
bool_df = df.applymap(lambda x: len(x) < 4)
bool_df
columns_numbers = [x for x in range(df.shape[1])]

to_remove = []
for i in columns_numbers:
    col_len = len(bool_df.iloc[:, i])
    # print(col_len // 2)
    # print(sum(bool_df.iloc[:, i]))
    if sum(bool_df.iloc[:, i]) > col_len // 2:
        to_remove.append(i)

if to_remove:
    for e in to_remove:
        columns_numbers.remove(e)

df.iloc[:, columns_numbers]





,"Федеральное государственное бюджетное учреждение «Федеральное \nагентство по правовой защите результатов интеллектуальной \nдеятельности военного, специального и двойного назначения».1","Федеральное государственное бюджетное учреждение «Федеральное \nагентство по правовой защите результатов интеллектуальной \nдеятельности военного, специального и двойного назначения».2","Федеральное государственное бюджетное учреждение «Федеральное \nагентство по правовой защите результатов интеллектуальной \nдеятельности военного, специального и двойного назначения».3"
0,Федеральное государственное бюджетное учрежден...,Федеральное государственное бюджетное учрежден...,Федеральное государственное бюджетное учрежден...
1,Новиков \nАлександр \nНиколаевич,Директор,"197 812,00"
2,Романов Алексей \nГеннадьевич,Заместитель директора,"184 153,00"
3,Рыбалкин \nВладимир \nГеннадьевич,Заместитель директора,"184 231,00"
4,Рукин Олег \nАлександрович,Заместитель директора,"90 675,00"
5,Маркова Ирина \nБорисовна,Главный бухгалтер,"177 429,00"


In [1]:
from number_to_text import num2text

num2text(1)


ModuleNotFoundError: No module named 'number_to_text'

In [2]:


class DataCleaner:
    """убирает лишние данные"""

    @staticmethod
    def remove_unwanted_symbols(df):        
        # TODO: чистка всех колонок
        df = df.applymap(lambda x: str(x).replace('\n', ' '))
        df = df.applymap(lambda x: ' '.join(str(x).split()))
        
        return df
    
    @staticmethod
    def remove_unwanted_cells(df):
        print('КОЛОНКИ В remove_unwanted_cells ----- ', df.columns)
        
        # убирает ячейки с нумерацией
        # print('--- DataCleaner.remove_unwanted_cells ---', df.columns)
        #TODO: почему тут только должность?

        # df = df[~df['position'].astype(str).str.isdigit()]
        return df
    
    
    @staticmethod
    def remove_unwanted_rows(df: pd.DataFrame):
        headers_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|\
                рублей|руб|cреднемесячная|зарпл.|плат[ы, а, е]|заработн[ой, ая]|плат[е, а, ы]|\
                    cреднемесячн[ая, ой]|зарплат[а, е, ной, ы]должност[ь, и, ей]|предприяти[е,я]|учреждени[е,я]|юридическое лицо)'

        to_delete = []
        temp_df = df.copy()
        cols_we_need = ['department', 'position', 'name', 'salary', 'page']
        temp_df = temp_df[[c for c in temp_df.columns if c in cols_we_need]]
        temp_df = temp_df.applymap(lambda x: str(x).lower())


        for row in temp_df.itertuples():
            index = row[0]
            headers_in_row = sum([bool(re.search(pattern=headers_pattern, string=str(e))) for e in list(row)])
            if headers_in_row > 2:
                to_delete.append(index)

            if len([e for e in set(row[1:]) if e and len(str(e)) > 2]) < 2:
                to_delete.append(index)

        df.drop(index=to_delete, axis=1, inplace=True)
        return df

    @staticmethod
    def remove_short_cols(df: pd.DataFrame):
        pass
        to_drop = []
        # df = df.applymap(str)
        # for i in len(df.columns):
        #     mean_cell_lenght = statistics.mean(list(map(len, df.iloc[:,i].values)))
        #     if mean_cell_lenght < 4:
        #         to_drop.append(i)
        
        # return df.drop()

    @staticmethod
    def remove_short_rows(df):
        # удаляет ряды с недостаточными данными
        # ! должно применяться после выбора норм колонок
        to_remove = []
        for tup in df.itertuples():
            res = [len(str(e)) for e in tup]
            if statistics.mean(res) < 5:
                to_remove.append(tup.Index)
        
        df.drop(index=to_remove, inplace=True)
        return df


    @staticmethod
    def concat_same_cols(df: pd.DataFrame):
        def sjoin(x): return ' '.join(x[x.notnull()].astype(str))
        return df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))

    @staticmethod
    def salary_parser(salary:str)->int:
        salary = str(salary)
        salary = salary.split(',')[0]
        salary = salary.split('.')[0]
        salary = salary.replace(' ', '')
        salary = ''.join([c for c in salary if c.isdigit()])

        if salary.isalnum():    
            return int(salary) * 12 

        return salary
    

    @staticmethod
    def get_numeric_salary(df: pd.DataFrame):
        # иногда в salary попадают колонки с описанием 
        
        cols = list(df.columns)
        if cols.count('salary') <= 1:
            return df

        def drop_not_numeric_salary(df) -> int:
            # берем только salary где большая часть - цифры

            max_val = 0
            realy_salary_col = 0
            for i in range(len(df.columns)):
                
                num_of_numeric_vals = sum(df.iloc[:,i].values)
                if num_of_numeric_vals > max_val:
                    realy_salary_col=i

            return realy_salary_col 
            
        salary_subset = df['salary'].copy()
        if type(salary_subset) == pd.DataFrame:
            bool_salary_subset = salary_subset.applymap(lambda x: str(x).isdigit())
        elif type(salary_subset) == pd.Series:
            bool_salary_subset = salary_subset.apply(lambda x: str(x).isdigit())
 
 #       bool_salary_subset = salary_subset.applymap(lambda x: str(x).isdigit())
 
        salary_col = drop_not_numeric_salary(bool_salary_subset)
        salary_subset = salary_subset.iloc[:, salary_col]
        # salary_subset.to_excel('salary_subset.xlsx')
        df['salary'].drop(columns=['salary'], inplace=True)
        #df.to_excel('df.xlsx')
        df.reset_index(inplace=True, drop=True)
        salary_subset.reset_index(inplace=True, drop=True)
        df['salary'] = salary_subset
        df = df.T.groupby(level=0).first().T # удалили дубликаты
              
        return df
        
    @staticmethod
    def sjoin(x): return ';'.join(set(x[x.notnull()].astype(str)))


    def clean_df(self, df):
        df = self.remove_unwanted_symbols(df)
        raw_salary_subset = df[['salary']]
        df['raw_salary'] = raw_salary_subset.groupby(level=0, axis=1).apply(lambda x: x.apply(self.sjoin, axis=1))
        
        if type(df['salary']) == pd.DataFrame: 
            df['salary'] = df['salary'].applymap(self.salary_parser)
        elif type(df['salary']) == pd.Series:
            df['salary'] = df['salary'].apply(self.salary_parser)
           
        df = self.get_numeric_salary(df)
        df = self.remove_unwanted_rows(df)
        # df = self.remove_unwanted_cells(df)
        # df = self.remove_short_rows(df)
        # df = self.concat_same_cols(df)
        return df



In [45]:
class ExcelParser:

    def __init__(self):
        self.data_cleaner = DataCleaner()
        self.cols_we_need = ['department', 'position', 'name', 'salary', 'sheet_name', 'documentfile_id']

    @staticmethod
    def find_ok_cols(cols: tuple) -> Union[dict[int, str], None]: # отдает словарь с колонками, которые надо переименовать
        '''ищем заголовки в таблице'''
        
        cols = list(map(str, cols))[:-1]
        cols = list(map(str.lower, cols))
        if len(set(cols)) == 1:
            return 

        ok_cols = 0
        result = {} # 'n_of_col':'new_name'
        for i, col in enumerate(cols):
            if len(result) > 5 or i > 20:
                return result

            if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col) and 'name' not in result.values():
                result[i] = 'name'
                ok_cols+=1

            elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, е, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col): #and "salary" not in result.values():
                result[i] = "salary"
                ok_cols+=1
                
            elif re.search(pattern='(должност[ь, и, ей])', string=col) and 'position' not in result.values(): 
                result[i] = 'position'
                ok_cols+=1
                
            elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическ|организаци|наименование [оу, мо])', string=col) and 'department' not in result.values():
                result[i] = 'department'
                ok_cols+=1

        if len(result) >= 2:
            return result
        return False
        

    def rename_cols(self, df: pd.DataFrame, new_cols: dict[int, str]):
        for k,v in new_cols.items():
            
            # if v == 'salary':
            #     df.rename(columns={df.columns[k]: 'salary_' + df.columns[k] }, inplace = True)
            # else:
            df.rename(columns={df.columns[k]: v}, inplace = True)
        return df 
    
    @staticmethod # table splitter
    def find_department_in_table(df: pd.DataFrame) -> list[dict[int, str]]: # [index, department]
        departments_n_indexes = []
        for row in df.itertuples():
            index = row[0]
            row = list(row)[1:-1]
            row = [e for e in row if len(str(e)) > 4]
            if len(set(row))<2:
                if not all([type(e) in [int, float] for e in row]):  
                    if statistics.mean([len(e) for e in row]) > 4:
                        departments_n_indexes.append({'index':index, 'dep': row[0]})

        return departments_n_indexes


        # departments_n_indexes = []
        # for row in df.itertuples():
        #     index = row[0]
        #     row = list(row)[1:]
                    
        #     if len(set(row))<2:
        #         if not all([type(e) in [int, float] for e in row]):
        #             if statistics.mean([len(e) for e in row]) > 4:
        #                 departments_n_indexes.append({'index':index, 'dep': row[0]})
        
        # return departments_n_indexes
    
    @staticmethod            
    def add_department_info_to_df(df: pd.DataFrame, dep_info: list[dict[int, str]]) -> pd.DataFrame:
        df['department'] = None
        for data in dep_info: 
            df.at[data['index'], 'department'] = data['dep']       
        df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=0)
        return df


    def detect_headers(self, df: pd.DataFrame)-> pd.DataFrame:
        for row in df.itertuples():
            index = row[0] 
            new_cols = self.find_ok_cols(row[1:])
            if new_cols:
                break

        if not new_cols:
            raise ValueError('не нашли заголовки таблицы')
            #return 'не нашли заголовки'
        
        #убрали все что ДО норм колонок  
        
        if 'department' not in new_cols.values(): 
            # если нет колонки с учреждениями - идем искать в таблице 
            dep_info = self.find_department_in_table(df)

            if dep_info:
                df = self.add_department_info_to_df(df, dep_info)
            else:
                merged_cells = ' '.join([e for e in set(list(map(str,sum(list(list(e) for e in df.iloc[:index,:].values),[])))) if 'nan' not in e and 'Unnamed' not in e and not e.isalnum() and e not in self.cols_we_need])
                cols = ' '.join ([e for e in set(list(map(str, list(df.iloc[:index,:].columns)))) if 'nan' not in e and 'Unnamed' not in e and not e.isalnum() and e not in self.cols_we_need])
                df['department'] = cols + ' ' + merged_cells
        df = df.iloc[index+1:,:]

        df = self.rename_cols(df, new_cols)
        
        return df

    
    def open_xlsx(self, filename: Path) -> Union[list[pd.DataFrame], None]:
        wb = load_workbook(filename) 
        sheets = wb.sheetnames
        dfs = []
        for sheet in sheets:
            df = pd.read_excel(filename, sheet_name=sheet, header=None)
            if not df.empty:
                df['sheet_name'] =  sheet
                dfs.append(df)
                
        if not dfs:
            raise ValueError(f'Не нашли таблиц в {filename}')
        return dfs

    def open_xls(self, filename: Path) -> list[pd.DataFrame]:
        wb = xlrd.open_workbook(filename, ignore_workbook_corruption=True)
        sheets = wb.sheet_names()
        dfs = []

        for sheet in sheets:
            df = pd.read_excel(filename, sheet_name=sheet, header=None)
            if not df.empty:
                df['sheet_name'] =  sheet
                dfs.append(df)

        if not dfs:
            raise ValueError(f'Не нашли таблиц в {filename}')
        return dfs
    

    def parse_file(self, filename: Path):
        
        try:
            if str(filename).endswith('.xlsx'):
                dfs = self.open_xlsx(filename)
            elif str(filename).endswith('.xls'):        
                dfs = self.open_xls(filename)
            else:
                raise ValueError('Расширение файла должно быть .xlsx или .xls')
        
        except Exception as ex:
            print(ex)

        parsed_dfs = []
        
        for df in dfs:
            df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=1)                
            df = df.fillna(method='ffill', axis=0)
            df = df.fillna(method='ffill', axis=1)
            # return df
            df = self.detect_headers(df)


            df = df[[c for c in df.columns if c in self.cols_we_need]]
            df = self.data_cleaner.clean_df(df)
            
            # слить имена 
            #df = df.T.groupby(level=0).first().T # удалили дубликаты
        
            '''def sjoin(x): return ' '.join(x[x.notnull()].astype(str))
            return df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))'''

            df['documentfile_id'] = filename.name.split('_')[0]
            parsed_dfs.append(df)
        try:
            return pd.concat(parsed_dfs).reset_index(drop=True)
        except:
            return parsed_dfs


file = r"D:\PROGR\LEARN_PYTHON\Declarator\declaratio        
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\processed\83337_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
excel_parser = ExcelParser()


#TODO: дропнуть ряды дубликаты
# temp = excel_parser.open_xlsx(Path(file))#[0]
# # temp[0].fillna(method='ffill', axis=1)
# temp[0].head(30)
df = excel_parser.parse_file(Path(file))
# df.to_excel('temp.xlsx')



#     my_dic['persons'].append(person)

#     my_dic    
# res = excel_parser.parse_file(Path(file))
#print(res)



,position,name,salary,sheet_name,department,raw_salary,documentfile_id
0,Заместитель директора,Пестряков Владислав Вячеславович,1670868,Лист1,Федеральное государственное бюджетное учрежден...,139239.75,83337
1,Заместитель директора по экологии,Кашуба Татьяна Геннадьевна,1600032,Лист1,Федеральное государственное бюджетное учрежден...,133336,83337
2,Главный бухгалтер,Конивец Ольга Викторовна,1534716,Лист1,Федеральное государственное бюджетное учрежден...,127893.29,83337
3,Директор,Чернышев Андрей Юрьевич,1364748,Лист1,Федеральное государственное бюджетное учрежден...,113729.53,83337
4,Заместитель директора,Панкратов Андрей Анатольевич,1110768,Лист1,Федеральное государственное бюджетное учрежден...,92564.33,83337
...,...,...,...,...,...,...,...
62,Заместитель директора,Шведов Сергей Андреевич,705168,Лист1,Федеральное государственное бюджетное учрежден...,58764,83337
63,Главный бухгалтер,Михайлова Полина Валерьевна,696288,Лист1,Федеральное государственное бюджетное учрежден...,58024,83337
64,Временно исполняющий обязанности директора (по...,Радюшкин Юрий Григорьевич,1741932,Лист1,Федеральное государственное бюджетное учрежден...,145161.42,83337
65,Заместитель директора по науке,Радюшкин Юрий Григорьевич,1310952,Лист1,Федеральное государственное бюджетное учрежден...,109246.97,83337


In [27]:
import pandas as pd

# f1 = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\table_to_concat_0.xlsx"
# f2 = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\table_to_concat_1.xlsx"

f1 = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\17_concat_atempt.xlsx"
f1 = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\18_concat_atempt.xlsx"

df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2)

df1.rename(columns={'position.1':'position'}, inplace=True)
df2.rename(columns={'position.1':'position'}, inplace=True)

# print(df1.columns)
# print(df2.columns)

def sjoin(x): return ';'.join(set(x[x.notnull()].astype(str)))

# df1.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))
# df2.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))






Index(['Unnamed: 0', 'name', 'page', 'salary', 'position', 'position',
       'documentfile_id'],
      dtype='object')
Index(['Unnamed: 0', 'name', 'salary', 'position', 'position',
       'documentfile_id'],
      dtype='object')


In [96]:

def fun(df):
    current_sheet = ''
    my_json = {}

    
    my_json['documents'] = []

    my_json['persons'] = []
    
    persons = []
    for e in res.itertuples():
        person = {
            'person':{
                'name': e.name,
                'role': e.position,
                'department' : e.department,
                'incomes': [e.salary],
                'raw_income': e.raw_salary
            }
        }

        if e.sheet_name != current_sheet:
            current_sheet = e.sheet_name
            document = {
                "documentfile_id" : e.documentfile_id,
                "sheet_title" : e.sheet_name,
            }  
            my_json['documents'].append(document)
            my_json['persons'].append(persons)

        persons.append(person)

    return my_json


my_json = fun(res)
#my_json#['persons'][0]

In [438]:
d = {
  "properties": {
    "persons": {
      "$id": "/properties/persons",
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "person": {
            "$id": "/properties/persons/properties/person",
            "type": "object",
            "properties": {
              "name": {
                "$id": "/properties/persons/properties/person/properties/name",
                "type": "string",
                "title": "Фамилия Имя Отчество (или с формате \"Фамилия И.О.\")",
                "default": "",
                "examples": [
                  "Бродский Михаил Наумович",
                  "Бродский М. Н.",
                  "Бродский М.Н."
                ]
              },
              "name_raw": {
                "$id": "/properties/persons/properties/person/properties/name_raw",
                "type": "string",
                "title": "Фамилия Имя Отчество (сырые данные из документа, указывать точно как было)",
                "default": "",
                "examples": [
                  "Бродский МихаиллНаумович",
                  "Бродский М...Н.",
                  "БродскийМ.Н."
                ]
              },
              "role": {
                "$id": "/properties/persons/properties/person/properties/role",
                "title": "Должность",
                "items": {
                  "examples": [
                    "Представитель Губернатора Санкт-Петербурга в Законодательном Собрании Санкт-Петербурга"
                  ]
                }
              },
              "department": {
                "$id": "/properties/persons/properties/person/properties/department",
                "title": "Департамент в органе власти",
                "examples": [
                  "Отдел по борьбе с коррупцией"
                ]
              }
            },
          },
          "incomes": {
            "$id": "/properties/persons/properties/incomes",
            "items": {
              "$id": "/properties/persons/properties/incomes/items",
              "type": "object",
              "properties": {
                "size": {
                  "$id": "/properties/persons/properties/incomes/items/properties/size",
                  "type": [
                    "number",
                    "null"
                  ],
                  "title": "Объём дохода в рублях",
                  "default": 0,
                  "examples": [
                    1474895
                  ]
                },
                "comment": {
                  "$id": "/properties/persons/properties/incomes/items/properties/comment",
                  "type": [
                    "string",
                    "null"
                  ],
                },
                },
            
          },

          "document_position": {
            "$id": "/properties/persons/properties/document_position",
            "type": "string",
            "title": "Адрес ячейки XLSX, номер страницы DOCX или другая информация для поиска записи в оригинальном документе"
          },
          "person_index": {
            "$id": "/properties/persons/properties/person_index",
            "type": "number",
            "title": "Порядковый номер в данном документе (если был указан в отдельном столбце)"
          }
        },
        "required": [
          "person"
        ]
      }
    },


    "document": {
      "$id": "/properties/document",
      "type": "object",
      "properties": {
        "documentfile_id": {
          "$id": "/properties/document/properties/documentfile_id",
          "type": ["number", "null"],
          "title": "ID файла в базе Декларатора",
          "default": "null",
          "examples": [
            35304
          ]
        },
        "archive_file": {
          "$id": "/properties/document/properties/archive_file",
          "type": "string",
          "title": "Полный путь к данном файлу внутри архива",
          "description": "Только для DocumentFile типа zip, где было несколько файлов внутри",
          "examples": [
            "Сотрудники министерства.xlsx",
            "руководители/2013/уточненный.docx"
          ]
        },
        "sheet_number": {
          "$id": "/properties/document/properties/sheet_number",
          "type": "number",
          "title": "Индекс листа (только для Excel)",
          "examples": [
            0, 1, 3
          ]
        },
        "sheet_title": {
          "$id": "/properties/document/properties/sheet_title",
          "type": "string",
          "title": "Заголовок листа (только для Excel)",
          "description": "Используется в случае, если где-то в шапке есть наименование лиц перечисленных на данном листе",
          "default": "null",
          "examples": [
            "Сведения о доходах, расходах, об имуществе и обязательствах имущественного характера, представленных федеральными государственными гражданскими служащими Министерства Российской Федерации по развитию Дальнего Востока, за период с 1 января 2015 г. по 31 декабря 2015 г."
          ]
        },
        "type": {
          "$id": "/properties/document/properties/type",
          "type": "string",
          "title": "Тип декларации",
          "default": "",
          "examples": [
            "Антикоррупционная декларация"
          ]
        },
        "name": {
          "$id": "/properties/document/properties/name",
          "type": "string",
          "title": "Название документа",
          "default": "",
          "examples": [
            "Сведения о доходах, об имуществе и обязательствах имущественного характера Губернатора Санкт-Петербурга, а также его супруги за период с 1 января 2016 года по 31 декабря 2016 года"
          ]
        },
        "file_size": {
          "$id": "/properties/document/properties/file_size",
          "type": "number",
          "title": "Размер файла документа",
          "default": "null"
        },
      
        "year": {
          "$id": "/properties/document/properties/year",
          "type": ["integer", "null"],
          "title": "Год декларируемых доходов",
   },
      
      
      }


    }
  }
}
}








In [54]:
folders = [Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx'), 
Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls')]
# "D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83367_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
#"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\parser\cool_excel\88007_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx.xlsx"
#"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\93918_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
# "D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\179826_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
#"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\186034_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx"
#"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\186393_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xls"
#errors = []
processed = [Path(r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\parser\cool_excel\можно что то сделать\не сплитится по учреждению")]
raw = Path(r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\processed")


ВСЕ РАБОТАЕТ, НАДО ТОЛЬКО ЗАПУСТИТЬ

for folder in folders:
    for file in os.listdir(folder):
        
        try:
            res = excel_parser.parse_file(folder / file)
            
            if type(res) == pd.DataFrame:
                res.to_excel('splited/' + file + '.xlsx')
            elif type(res) == list:
                for i, e in enumerate(res):
                    e.to_excel(f'splited/{i}__' + e + '.xlsx')
                    
        except Exception as ex:
            print(ex)
            errors.append({'file':file, 'error':traceback.print_exc()})
                print(ex)



#             res = excel_parser.parse_file(folder / file)
#         except Exception as ex:
#             print(file)
#             print(ex)
        #res.to_excel(f'cool_excel/{file}.xlsx')

            

In [5]:
errors

[{'file': '102020_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).xlsx',
  'error': None},
 {'file': 'processed', 'error': None},
 {'file': '~$105387_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': None},
 {'file': '99506_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xls',
  'error': None},
 {'file': 'processed', 'error': None},
 {'file': 'не видел заголовки', 'error': None}]

In [94]:

re.search(pattern='организаци', string='Наименование организации')


<re.Match object; span=(13, 23), match='организаци'>

In [31]:
pd.concat([df1, df2])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects